In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))
import talib
import sqlite3 as sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import matplotlib.font_manager as fm
from mplfinance.original_flavor import candlestick_ohlc
from util.util import get_ucodes

d:\pycharmprojects\scrapy-001\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\pycharmprojects\scrapy-001\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
d:\pycharmprojects\scrapy-001\venv\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
print(path_sqlite)
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()
conn.row_factory = lambda cursor, row: row[0]
no_days = 1000

D:\PycharmProjects\scrapy-001\data\aastock\sqlite\hk-marketwatch.db


In [3]:
data1 = {}
ucodes = ['hsi', 'hsce', 'hstech', 'ssec', 'csi300', 'sp500', 'dji', 'ixic']
for ucode in ucodes:
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC LIMIT {}) AS t 
                            ORDER BY t.stime """.format(ucode, no_days)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data1[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
    data1[ucode].index = pd.to_datetime(data1[ucode].sdate)
    data1[ucode]['weekday'] = data1[ucode].index.dayofweek
    data1[ucode]['pchng'] = data1[ucode]['last'].pct_change()*100
conn.close()

In [4]:
data2 = {}
data4 = {}
for ucode, df in data1.items():
    data2[ucode] = {}
    data4[ucode] = pd.DataFrame(columns=['Mon', 'Tue', 'Wed', 'Thu', 'Fri'])
    for i in [1, 2, 3, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]:
        data3 = []
        for j in range(5):
            no = df.loc[(df.weekday == j)].shape
            if no[0] >= i:
                pchng = "{:.2f}".format(round(df.loc[(df.weekday == j)]['pchng'].tail(i).mean(), 2))
                plast = "{:.2f}".format(round(df.loc[(df.weekday == j)]['last'].tail(i).mean(), 2))
                sdates = ','.join(df.loc[(df.weekday == j)]['sdate'].tail(i).values.tolist())
            else:
                pchng = ""
                plast = ""
                sdates = ""
            data3.append(str(pchng)+','+str(plast)+','+sdates)
        data2[ucode][i] = data3

html = ''
for k1, v1 in data2.items():
    html += '<tr class="thead-light"><td class="h5" colspan="6">'+k1+'</td></tr>'
    html += '<tr><td>Week</td> <td>Mon</td> <td>Tue</td> <td>Wed</td> <td>Thu</td> <td>Fri</td></tr>'
    for k2, v2 in v1.items():
        html += '<tr><td>'+str(k2)+'</td>'
        for v3 in v2:
            v3_1 = v3.split(',')
            if v3_1[0] == "":
                html += '<td> / </td>'
            elif float(v3_1[0]) <= 0:
                html += '<td><span class="text-success h6">('+str(v3_1[0])+') '+v3_1[1]+'</span> &nbsp&nbsp <span class="text-white">'+v3_1[2]+'</span></td>'
            elif float(v3_1[0]) > 0:
                html += '<td><span class="text-danger h6">(+'+str(v3_1[0])+') '+v3_1[1]+'</span> &nbsp&nbsp <span class="text-white">'+v3_1[2]+'</span></td>'
        html += '</tr>'

html2 = """
<html>
<head>
<title>轮证概率</title>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">
<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.5.1/jquery.min.js" integrity="sha512-bLT0Qm9VnAYZDflyKcBaQ2gg0hSYNQrJ8RilYldYQ1FxQYoCLtUjuuRuZo+fjqhx/qtq/1itJ0C2ejDxltZVFg==" crossorigin="anonymous"></script>
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" integrity="sha384-JcKb8q3iqJ61gNV9KGb8thSsNjpSL0n8PARn9HuZOnIxN0hoP+VmmDGMN5t9UJ0Z" crossorigin="anonymous">
</head>
<style>
td {font-size: 12px;}
</style>
<body class="bg-dark">
<div class="container">

<div class="row">
<div class="col-sm-12">
<table border="0" class="table table-sm table-dark table-striped">"""+html+"""</table>
</div>
</div>

</div>
</body>
</html>
"""

path = os.path.join(path_root2, 'algo', '轮证概率.html')
writer = open(path, "w", encoding="utf-8")
writer.write(html2)
writer.close()